# Tree-based classifier

Include all possible dimensions and train a treebased classifier

In [1]:
import pandas as pd

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
from sklearn import utils
from sklearn.tree import DecisionTreeRegressor

## Load Data

In [63]:
df = pd.read_csv('/tmp/rideindego_all.csv')

/Users/leima/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,3,4,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
df['date'] = pd.to_datetime(df.date)

In [65]:
df.head()

,bike_id,bike_type,duration,end_lat,end_lon,end_station_id,end_time,passholder_type,plan_duration,start_lat,start_lon,start_station_id,start_time,trip_id,trip_route_category,date,hour,weekday,month
0,3640,standard,4.0,39.9384,-75.173271,3064,7/1/2016 0:11,Indego30,30,39.946331,-75.1698,3063.0,7/1/2016 0:07,25183529,One Way,2016-07-01,0,4,7
1,3349,standard,8.0,39.94138,-75.145638,3026,7/1/2016 0:18,Indego30,30,39.95295,-75.139793,3124.0,7/1/2016 0:10,25183528,One Way,2016-07-01,0,4,7
2,5279,standard,17.0,39.933151,-75.162483,3034,7/1/2016 0:28,Indego30,30,39.93082,-75.174744,3043.0,7/1/2016 0:11,25183527,One Way,2016-07-01,0,4,7
3,5198,standard,25.0,39.94138,-75.145638,3026,7/1/2016 0:38,Walk-up,0,39.94138,-75.145638,3026.0,7/1/2016 0:13,25183526,Round Trip,2016-07-01,0,4,7
4,3382,standard,45.0,39.947109,-75.166183,3010,7/1/2016 0:58,Walk-up,0,39.954239,-75.161377,3037.0,7/1/2016 0:13,25183525,One Way,2016-07-01,0,4,7


In [66]:
df = df[df.bike_type == 'standard']
coi = [
    'passholder_type', 
    'trip_route_category', 'hour', 'weekday', 'month'
]
cot = ['duration']
df = df[coi+cot]

In [77]:
df.head()

,passholder_type,trip_route_category,hour,weekday,month,duration
0,Indego30,One Way,0,4,7,4.0
1,Indego30,One Way,0,4,7,8.0
2,Indego30,One Way,0,4,7,17.0
3,Walk-up,Round Trip,0,4,7,25.0
4,Walk-up,One Way,0,4,7,45.0


In [67]:
for i in df.columns:
    print(
        i, ': ',
        df[i].isnull().any()
    )

passholder_type :  False
trip_route_category :  False
hour :  False
weekday :  False
month :  False
duration :  False


In [68]:
scaler = StandardScaler()

In [145]:
X = df[coi]
#X = pd.get_dummies(X)
y = df[cot]
y = scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3, random_state=42
    #,stratify=Y
)

In [146]:
class MultiColumnLabelEncoder:
    def __init__(self, encoders = None, columns = None):
        self.columns = columns # array of column names to encode
        if encoders:
            self.encoders = encoders
        else:
            self.encoders = {}

    def fit(self,X,y=None):
        self.check_encoders = []
        return self # not relevant here

    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                _le = LabelEncoder()
                if self.encoders.get(col):
                    output[col] = self.encoders.get(col).transform(output[col])
                else:
                    output[col] = _le.fit_transform(output[col])
                    print(f'1. preparing encoder for {col}')
                    self.encoders[col] = _le
                    self.check_encoders.append({col: _le})
        else:
            for colname,col in output.iteritems():
                _le = LabelEncoder()
                if self.encoders.get(col):
                    output[colname] = self.encoders.get(col).transform(col)
                else:
                    output[colname] = _le.fit_transform(col)
                    print(f'2. preparing encoder for {col}')
                    self.encoders[col] = _le
                    self.check_encoders.append({col: _le})
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [147]:
encoders_obj = MultiColumnLabelEncoder(columns=coi)
encoders_obj.fit_transform(df)
dt_encoders = encoders_obj.encoders

1. preparing encoder for passholder_type
1. preparing encoder for trip_route_category
1. preparing encoder for hour
1. preparing encoder for weekday
1. preparing encoder for month


In [148]:
coi

['passholder_type', 'trip_route_category', 'hour', 'weekday', 'month']

In [163]:
dt_regressor = GridSearchCV(
    DecisionTreeRegressor(random_state=42), cv=3, 
    param_grid={"max_depth": [1,2,3,4,5,6,7,8]}
)

pipeline_steps = [
    ('encoding', MultiColumnLabelEncoder(columns=coi, encoders=dt_encoders)),
    ('dt_classifier', dt_regressor)
    # add more pipeline steps as needed
]

In [164]:
model = Pipeline(pipeline_steps)

In [165]:
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('encoding', <__main__.MultiColumnLabelEncoder object at 0x1a2c45e828>), ('dt_classifier', GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           m...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [166]:
model.score(X_test, y_test)

0.03406175535828204

In [167]:
model.score(X_train, y_train)

0.03549311276539002

In [168]:
dt_regressor.best_params_

{'max_depth': 6}

In [275]:
df_test = X_test.sample(n=2)

In [276]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 516568 to 1065860
Data columns (total 5 columns):
passholder_type        2 non-null object
trip_route_category    2 non-null object
hour                   2 non-null int64
weekday                2 non-null int64
month                  2 non-null int64
dtypes: int64(3), object(2)
memory usage: 96.0+ bytes


In [277]:
model.predict(df_test)

array([-0.03740941,  0.32952739])

In [278]:
'{}'.format(model.predict(df_test))

'[-0.03740941  0.32952739]'

In [279]:
scaler.inverse_transform(model.predict(df_test))

array([20.121273  , 45.15476532])

In [280]:
df_test = X_test.sample(n=1)

In [283]:
df_test.to_json(orient='records')

'[{"passholder_type":"Indego30","trip_route_category":"One Way","hour":15,"weekday":3,"month":6}]'

In [287]:
record = json.loads('{"passholder_type":"Indego30","trip_route_category":"One Way","hour":15,"weekday":3,"month":6}')

In [288]:
df_test_recon = pd.DataFrame.from_records([record])

In [289]:
df_test_recon

,hour,month,passholder_type,trip_route_category,weekday
0,15,6,Indego30,One Way,3


## Playground: Ridge

In [102]:
from sklearn.model_selection import GridSearchCV

In [129]:
from sklearn.linear_model import Ridge

In [134]:
from sklearn.model_selection import cross_val_score

In [138]:
regr_ridge = GridSearchCV(
    Ridge(), cv=3, 
    param_grid={"alpha": [100, 20, 10,1,0.1]}
)

In [139]:
lr_pipeline = Pipeline(
    [('ridge', regr_ridge)]
)

In [140]:
lr_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('ridge', GridSearchCV(cv=3, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [100, 20, 10, 1, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [141]:
lr_pipeline.score(X_test, y_test)

0.03144391124409851

In [142]:
print(regr_ridge.best_params_)

{'alpha': 100}


In [143]:
cross_val_score(regr_ridge, X_train, y_train, cv=10)

array([0.03066379, 0.03202355, 0.03011045, 0.0321168 , 0.03300355,
       0.02993489, 0.03229209, 0.03010943, 0.031031  , 0.03156605])